In [39]:
from io import BytesIO
import requests
import time
import pandas as pd
import utils.misc as misc
from config import *
import yahoo_fin.stock_info as si
from yahooquery import Ticker
import quandl
import os
from datetime import datetime, timedelta

In [10]:
BASE_URL = r"https://www.alphavantage.co/query?"
av_api = 'CQLP7J9VZG14Y7O4'

In [11]:
def av_api(func, api_key, symbol):
    assert symbol is not None
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}function={func}&symbol={symbol}&apikey={api_key}"

    response = requests.get(url)
    if func == 'EARNINGS':
        return pd.DataFrame(response.json()['quarterlyEarnings'])
    elif func == 'BALANCE_SHEET':
        return pd.DataFrame(response.json()['quarterlyReports'])
    elif func == 'INCOME_STATEMENT':
        return pd.DataFrame(response.json()['quarterlyReports'])

In [33]:
def get_av_data(symbol):
    #For reported Earnings & Analyst Estimated EPS, totalRevenue, commonStockSharesOutstanding
    df1 = av_api('EARNINGS',av_api, symbol)
    df2 = av_api('INCOME_STATEMENT',av_api, symbol)[['fiscalDateEnding','totalRevenue']]
    df1 = df1.merge(df2, on='fiscalDateEnding', how='left')
    df3 = av_api('BALANCE_SHEET',av_api, symbol)[['fiscalDateEnding','commonStockSharesOutstanding']]
    df1 = df1.merge(df3, on='fiscalDateEnding', how='left')

    return df1

In [78]:
def get_ex_dates(symbol):
    symbol = 'META'
    tkr = Ticker(symbol)
    df = tkr.sec_filings
    df = df[(df['type'] == '10-Q') | ((df['type'] == '10-K'))]
    df.reset_index(inplace=True)
    df = df[['date','title']]
    df.rename(columns={'date': 'Ex-Date'}, inplace=True)
    return df

In [79]:
symbol = 'META'
df1 = get_av_data(symbol)
df2 = get_ex_dates(symbol)

In [81]:
df1

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,totalRevenue,commonStockSharesOutstanding
0,2022-12-31,2023-02-01,1.76,2.22,-0.46,-20.7207,31920000000,2639000000
1,2022-09-30,2022-10-26,1.64,1.89,-0.25,-13.2275,27710000000,2682000000
2,2022-06-30,2022-07-27,2.46,2.59,-0.13,-5.0193,28818000000,2704000000
3,2022-03-31,2022-04-27,2.72,2.56,0.16,6.25,27904000000,2725000000
4,2021-12-31,2022-02-02,3.67,3.84,-0.17,-4.4271,33527000000,2764000000
5,2021-09-30,2021-10-25,3.22,3.19,0.03,0.9404,29006000000,2814000000
6,2021-06-30,2021-07-28,3.61,3.03,0.58,19.1419,29073000000,2834000000
7,2021-03-31,2021-04-28,3.3,2.37,0.93,39.2405,26167000000,2847000000
8,2020-12-31,2021-01-27,3.88,3.22,0.66,20.4969,27940000000,2854000000
9,2020-09-30,2020-10-29,2.71,1.91,0.8,41.8848,21466000000,2850000000


##IGNORE FOR NOW

    NASDAQ DATALINK - FOR SHORT INTEREST

In [28]:
ndl_api = 'dys7Lqv_HziWD1ZFk4ry'

In [29]:
quandl.ApiConfig.api_key = ndl_api

In [30]:
os.chdir('/Users/yash/Desktop/Trading/intern/Basil/finra_short_interest')

In [31]:
df2 = pd.read_csv('equityshortinterest_2023_02_26_05_44_02.csv')

In [32]:
df2.head()

,Settlement Date,Issue Name,Symbol,Market,Current Short,Previous Short,Chg,% Change from Previous,Avg Daily Vol,Days to Cover,Revision Flag
0,2022-04-14,10X Capital Venture Acquisitio,VCXBU,NYSE,660,0,660,100.00,10175,1.00,NaN
1,2022-07-29,10X Capital Venture Acquisitio,VCXB,NYSE,9578,9390,188,2.00,2974,3.22,NaN
2,2022-05-31,10X Capital Venture Acquisitio,VCXBWS,NYSE,80934,3283,77651,2365.25,33603,2.41,NaN
3,2022-11-30,10X Capital Venture Acquisitio,VCXBWS,NYSE,150,150,0,0.00,0,999.99,NaN
4,2022-12-30,10X Capital Venture Acquisitio,VCXB,NYSE,8816,3948,4868,123.30,337962,1.00,NaN
